In [116]:
import numpy as np
import pandas as pd

In [117]:
#read Dataset
df = pd.read_csv("diabetes.csv")

In [118]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [119]:
#checking correlation with output
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

## Simple Model Without Hyperparameter

In [120]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [121]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [122]:
X = scaler.fit_transform(X)

In [123]:
X.shape

(768, 8)

In [124]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [125]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout


In [126]:
model = Sequential()
model.add(Dense(32,activation = 'relu',input_dim = 8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer = 'Adam',loss='binary_crossentropy',metrics=['accuracy'])



In [127]:
model.fit(X_train,y_train, batch_size=32,epochs=100,validation_data = (X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 12ms/step - loss: 0.6332 - accuracy: 0.6922 - val_loss: 0.6092 - val_accuracy: 0.7468
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5861 - accuracy: 0.7329 - val_loss: 0.5717 - val_accuracy: 0.7727
Epoch 3/100
20/20 [==============================] - 0s 8ms/step - loss: 0.5542 - accuracy: 0.7410 - val_loss: 0.5470 - val_accuracy: 0.7597
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5306 - accuracy: 0.7476 - val_loss: 0.5284 - val_accuracy: 0.7727
Epoch 5/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5136 - accuracy: 0.7541 - val_loss: 0.5130 - val_accuracy: 0.7727
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5003 - accuracy: 0.7573 - val_loss: 0.5036 - val_accuracy: 0.7727
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4907 - accuracy: 0.7606 - val_loss: 0.4959 - val_accuracy: 0.7792
Epoch 8/100


## Steps

In [128]:
# 1. How to select apppropriate optimizer
# 2. Number of nodes in a layer
# 3. How to select no. of middle layer
# 4. All in one model

In [129]:
pip install -U keras-tuner

In [130]:
import kerastuner as kt

## Hyperparameter Tuning On Optimizers

In [131]:
def build_model(hp):

  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics = ['accuracy'])

  return model


In [132]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5)

Reloading Tuner from ./untitled_project/tuner0.json


In [133]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

In [134]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [135]:
model = tuner.get_best_models(num_models = 1)[0]

In [136]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [137]:
model.fit(X_train,y_train,batch_size = 32 , epochs = 100 , initial_epoch = 6,validation_data = (X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5155 - accuracy: 0.7752 - val_loss: 0.4959 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5021 - accuracy: 0.7801 - val_loss: 0.4833 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4914 - accuracy: 0.7850 - val_loss: 0.4736 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4831 - accuracy: 0.7850 - val_loss: 0.4671 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4774 - accuracy: 0.7850 - val_loss: 0.4626 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4726 - accuracy: 0.7850 - val_loss: 0.4588 - val_accuracy: 0.7792
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4688 - accuracy: 0.7850 - val_loss: 0.4563 - val_accuracy: 0.7792
Epoch 14

## Hyperparameter Tuning on Number of nurons

In [138]:
def build_model(hp): #hp means hypermeter
  model = Sequential()
  units = hp.Int('units',min_value=8,max_value=128,step = 8)


  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))



  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics = ['accuracy'])

  return model

In [139]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5,
                        directory = 'mydir',
                        project_name = 'amit')

Reloading Tuner from mydir/amit/tuner0.json


In [140]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

In [141]:
tuner.get_best_hyperparameters()[0].values

{'units': 120}

In [142]:
model = tuner.get_best_models(num_models = 1)[0]

In [143]:
model.fit(X_train,y_train,batch_size = 32 , epochs = 100 , initial_epoch = 6,validation_data = (X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 11ms/step - loss: 0.5269 - accuracy: 0.7557 - val_loss: 0.5097 - val_accuracy: 0.7987
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4992 - accuracy: 0.7687 - val_loss: 0.4914 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4814 - accuracy: 0.7736 - val_loss: 0.4838 - val_accuracy: 0.7922
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4700 - accuracy: 0.7769 - val_loss: 0.4794 - val_accuracy: 0.7727
Epoch 11/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4634 - accuracy: 0.7801 - val_loss: 0.4763 - val_accuracy: 0.7597
Epoch 12/100
20/20 [==============================] - 0s 10ms/step - loss: 0.4588 - accuracy: 0.7801 - val_loss: 0.4737 - val_accuracy: 0.7792
Epoch 13/100
20/20 [==============================] - 0s 10ms/step - loss: 0.4553 - accuracy: 0.7801 - val_loss: 0.4741 - val_accuracy: 0.7792
Epoch 

## Hyperparameter tuning On Number of Layers

In [144]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(72,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layer',min_value =1,max_value=10)):

      model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics = ['accuracy'])

  return model


In [145]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5,
                        directory = 'mydir1',
                        project_name = 'Amit Bhardwaj')

Reloading Tuner from mydir1/Amit Bhardwaj/tuner0.json


In [146]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

In [147]:
tuner.get_best_hyperparameters()[0].values

{'num_layer': 8}

In [148]:
model = tuner.get_best_models(num_models = 1)[0]

In [149]:
model.fit(X_train,y_train,batch_size = 32 , epochs = 100 , initial_epoch = 6,validation_data = (X_test,y_test))

Epoch 7/100
20/20 [==============================] - 5s 32ms/step - loss: 0.4370 - accuracy: 0.7964 - val_loss: 0.4922 - val_accuracy: 0.7792
Epoch 8/100
20/20 [==============================] - 0s 13ms/step - loss: 0.4291 - accuracy: 0.7915 - val_loss: 0.4661 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 13ms/step - loss: 0.4158 - accuracy: 0.8111 - val_loss: 0.4731 - val_accuracy: 0.8052
Epoch 10/100
20/20 [==============================] - 0s 10ms/step - loss: 0.4011 - accuracy: 0.8208 - val_loss: 0.4834 - val_accuracy: 0.8052
Epoch 11/100
20/20 [==============================] - 0s 12ms/step - loss: 0.4106 - accuracy: 0.8094 - val_loss: 0.4559 - val_accuracy: 0.7922
Epoch 12/100
20/20 [==============================] - 0s 12ms/step - loss: 0.3911 - accuracy: 0.8208 - val_loss: 0.5314 - val_accuracy: 0.7792
Epoch 13/100
20/20 [==============================] - 0s 14ms/step - loss: 0.3761 - accuracy: 0.8404 - val_loss: 0.4564 - val_accuracy: 0.8182
Ep

## Hyperparameter Tuning All in One

In [150]:
def build_model(hp):
  model = Sequential()

  counter = 0


  for i in range(hp.Int('num_layer',min_value =1,max_value=10)):
    if counter == 0:

      model.add(
          Dense(
            hp.Int(('units') + str(i), min_value = 8,max_value = 128 , step = 8),
            activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid']),
            input_dim = 8
            )

          )
      model.add(Dropout(hp.Choice('dropout'+ str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
        model.add(
        Dense(
          hp.Int(('units') + str(i), min_value = 8,max_value = 128 , step = 8),
          activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid']),
          input_dim = 8
          )
        )
        model.add(Dropout(hp.Choice('dropout'+ str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    counter += 1

  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics = ['accuracy'])

  return model



In [151]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 3,
                        directory = 'final2',
                        project_name = 'Amit Bhardwaj 2')

Reloading Tuner from final2/Amit Bhardwaj 2/tuner0.json


In [152]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

In [153]:
tuner.get_best_hyperparameters()[0].values

{'num_layer': 8,
 'units0': 64,
 'activation0': 'relu',
 'dropout0': 0.1,
 'optimizer': 'rmsprop',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 8,
 'activation5': 'relu',
 'dropout5': 0.1,
 'units6': 8,
 'activation6': 'relu',
 'dropout6': 0.1,
 'units7': 8,
 'activation7': 'relu',
 'dropout7': 0.1}

In [154]:
model = tuner.get_best_models(num_models = 1)[0]

In [155]:
model.fit(X_train,y_train, epochs = 200 , initial_epoch = 6,validation_data = (X_test,y_test))

Epoch 7/200
20/20 [==============================] - 4s 21ms/step - loss: 0.6725 - accuracy: 0.6531 - val_loss: 0.6648 - val_accuracy: 0.6429
Epoch 8/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6462 - accuracy: 0.6531 - val_loss: 0.6309 - val_accuracy: 0.6429
Epoch 9/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6173 - accuracy: 0.6531 - val_loss: 0.5926 - val_accuracy: 0.6429
Epoch 10/200
20/20 [==============================] - 0s 7ms/step - loss: 0.5839 - accuracy: 0.6531 - val_loss: 0.5715 - val_accuracy: 0.6429
Epoch 11/200
20/20 [==============================] - 0s 7ms/step - loss: 0.5719 - accuracy: 0.6531 - val_loss: 0.5594 - val_accuracy: 0.6429
Epoch 12/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5723 - accuracy: 0.6531 - val_loss: 0.5506 - val_accuracy: 0.6429
Epoch 13/200
20/20 [==============================] - 0s 7ms/step - loss: 0.5572 - accuracy: 0.6531 - val_loss: 0.5436 - val_accuracy: 0.6429
Epoch 14